In [1]:
import pandas as pd
import pickle
import numpy as np
from sklearn.model_selection import LeaveOneOut
from sklearn.linear_model import Ridge
from sklearn.preprocessing import normalize

In [2]:
domains = ['brand', 'good', 'trait', 'food', 'occupation', 'risk', 'people']
dims = [('sincere','exciting'),
        ('hedonic','utilitarian'),
        ('masculine','feminine'),
        ('tasty','nutritious'),
        ('significance','autonomy'),
        ('dread-inducing','unknowable'),
        ('warm', 'competent')]

In [3]:
with open("data/dict_of_Xs.pickle", "rb") as handle:
    dict_of_Xs = pickle.load(handle)
    
with open("data/dict_of_ys.pickle", "rb") as handle:
    dict_of_ys = pickle.load(handle)

In [4]:
# with open("data/dict_of_Xs_nothing_dropped.pickle", "rb") as handle:
#     dict_of_Xs = pickle.load(handle)
    
# with open("data/dict_of_ys_nothing_dropped.pickle", "rb") as handle:
#     dict_of_ys = pickle.load(handle)

In [5]:
ridge = Ridge(alpha=10) # pre-reg'd analyses revealed ridge with alpha=10 had highest rsquared across dimensions 

In [14]:
for domain, dim_pair in zip(domains, dims):
    for dim in dim_pair:
        X = dict_of_Xs[domain].values
#         X = normalize(X, norm='l2') # ONLY LEAVE THIS UNCOMMENTED IF WANT VECTORS NORMALIZED TO UNIT LENGTH
        y = dict_of_ys[dim].values
        loo = LeaveOneOut()
        loo.get_n_splits(X) # not sure I actually need this?
        y_preds = np.zeros(shape=len(y))
        for train_index, test_index in loo.split(X):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
            ridge.fit(X=X_train, y=y_train)
            y_pred = ridge.predict(X=X_test)[0]
            y_preds[test_index] = y_pred
        preds_and_actual =  pd.DataFrame({f'Actual {dim}':y, f'Predicted {dim}':y_preds}, index=dict_of_ys[dim].index)
        preds_and_actual.to_csv(f'results/best_ridge_pred_vs_actual/{dim}.csv')
#         preds_and_actual.to_csv(f'results/best_ridge_pred_vs_actual_nothing_dropped/{dim}.csv')
#         preds_and_actual.to_csv(f'results/best_ridge_pred_vs_actual_normalized_vectors/{dim}.csv')